In [2]:
# ==============================================================
# 0. LIBRERÍAS Y CONFIGURACIÓN INICIAL
# ==============================================================

import os
import random
import math

import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score

import tensorflow as tf
from tensorflow import keras

# --------------------------------------------------------------
# Fijamos semillas para reproducibilidad básica
# --------------------------------------------------------------
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
random.seed(SEED)

# ==============================================================
# 1. CARGA DEL CONJUNTO DE DATOS LIMPIO
# ==============================================================

# Ruta al archivo limpio (ajusta el nombre si es necesario)
DATA_PATH = "listings_model_no_outliers.csv"

# Leemos el CSV en un DataFrame
df = pd.read_csv(DATA_PATH)

# Mostramos información básica para verificar que cargó bien
print("Tamaño dataframe:", df.shape)
print("Primeras columnas:", df.columns.tolist()[:20])

Tamaño dataframe: (40550, 147)
Primeras columnas: ['id', 'latitude', 'longitude', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'price', 'minimum_nights', 'maximum_nights', 'number_of_reviews', 'reviews_per_month', 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value', 'neighbourhood_cleansed_Barnet']


In [4]:
# ==============================================================
# 2. DEFINICIÓN DE LA VARIABLE OBJETIVO 
# ==============================================================

# En el proyecto queremos predecir el "precio por noche".
# Buscamos el nombre de la columna de target.
posibles_targets = ["price", "price_night", "price_per_night"]
target_col = None

for col in posibles_targets:
    if col in df.columns:
        target_col = col
        break

if target_col is None:
    raise ValueError("No se encontró ninguna columna de precio en el DataFrame. "
                     "Revisa el nombre de la columna de precio y actualiza 'posibles_targets'.")

print(f"Usaremos la columna '{target_col}' como variable objetivo (y).")

# Si la columna de precio está como texto con símbolos ($, comas), la convertimos a numérico
if df[target_col].dtype == "O":
    df[target_col] = (
        df[target_col]
        .astype(str)
        .str.replace(r"[\$,]", "", regex=True)
        .astype(float)
    )

# Nos aseguramos de que no haya valores nulos en la variable objetivo
df = df[df[target_col].notna()].copy()

# ==============================================================
# 3. SELECCIÓN DE VARIABLES PREDICTORAS (X)
# ==============================================================

# a) Eliminamos columnas identificadoras o claramente no útiles para la red
cols_descartar = [
    target_col,
    "id",
    "listing_id",
    "name",
    "description",
    "host_name",
    "host_id",
    "last_review"
]

cols_descartar = [c for c in cols_descartar if c in df.columns]

# Tomamos todas las demás columnas como candidatos
X_raw = df.drop(columns=cols_descartar)

# b) Nos quedamos solo con variables numéricas (incluye dummies one-hot)
X_num = X_raw.select_dtypes(include=[np.number]).copy()

print("Variables numéricas iniciales:", X_num.shape[1])

# --- c) Detectar columnas sin variación (todas las filas tienen el mismo valor) ---
cols_sin_variacion = X_num.columns[X_num.nunique() <= 1].tolist()
print("Columnas sin variación (se eliminan):", len(cols_sin_variacion))

# --- d) Detectar dummies casi siempre 0 o casi siempre 1 ---
# Identificamos columnas binarias (solo 0 y 1, ignorando NaN)
binary_cols = []
for col in X_num.columns:
    valores = set(X_num[col].dropna().unique())
    if valores.issubset({0, 1}):
        binary_cols.append(col)

# Calculamos proporción de 1s en cada dummy
proporcion_positivos = X_num[binary_cols].mean()

# Umbral: menos de 1% de 1s (o más de 99% de 1s)
umbral = 0.01
cols_casi_todo_cero = proporcion_positivos[proporcion_positivos < umbral].index.tolist()
cols_casi_todo_uno = proporcion_positivos[proporcion_positivos > (1 - umbral)].index.tolist()

print("Dummies casi siempre 0 (se eliminan):", len(cols_casi_todo_cero))
print("Dummies casi siempre 1 (se eliminan):", len(cols_casi_todo_uno))

# Lista final de columnas a eliminar por poca información
cols_eliminar_por_poca_info = list(set(cols_sin_variacion + cols_casi_todo_cero + cols_casi_todo_uno))

# Creamos la matriz final X eliminando esas columnas
X = X_num.drop(columns=cols_eliminar_por_poca_info).copy()

feature_names = X.columns.tolist()

print("Variables finales usadas como features:", len(feature_names))
print("Número de variables numéricas utilizadas como features:", len(feature_names))

# Variable objetivo (serie o vector)
y = df[target_col].values

Usaremos la columna 'price' como variable objetivo (y).
Variables numéricas iniciales: 17
Columnas sin variación (se eliminan): 2
Dummies casi siempre 0 (se eliminan): 0
Dummies casi siempre 1 (se eliminan): 2
Variables finales usadas como features: 15
Número de variables numéricas utilizadas como features: 15
